In [1]:
import pandas as pd
import math
from iapws import IAPWS97
%run class_module_optimization.ipynb

In [4]:
def print_f (node1, node2, pipe):
    print('{:14} {:>14} {:>14}'.format('', 'Inlet Node', 'Outlet Node'))
    print('{:14} [{:^14}] [{:^14}]'.format('', node1.ID, node2.ID))
    print('{:14} {:14.4f} {:14.4f}'.format('Pressure', node1.p, node2.p))
    print('{:14} {:14.4f} {:14.4f}'.format('Temperature', node1.t, node2.t))
    print('{:14} {:14.4f} {:14.4f}'.format('Mass Flow', node1.mf, node2.mf))
    print('{:14} {:14.4f} {:14.4f}'.format('Diameter', node1.d, node2.d))
    print('{:14} {:14.4f} {:14.4f}'.format('Velocity', node1.v, node2.v))
    print('{:14} {:14.4f}'.format('Reynolds Number', node1.reynolds))
    print('{:14} {:14.4f} {:14.4f}'.format('Enthalpy', node1.h, node2.h))
    print('{:14} {:14.4f} {:14.4f}'.format('SpecificExergy', node1.sp_exergy, node2.sp_exergy))
    print('{:14} {:14.4f} {:14.4f}'.format('Exergy (kW)', node1.exergy, node2.exergy))
    print('{:14} {:14.4f} {:14.4f}'.format('SiO2', node1.sio2, node2.sio2))
    print('{:14} {:14.4f} {:14.4f}'.format('CO2', node1.co2, node2.co2))
    print('{:14} {:14.4f} {:14.4f}'.format('H2S', node1.h2s, node2.h2s))

    if pipe.ID[0:3] == 'STL':
        print('{:14} {:14.4} {:14.4}'.format('NCG', node1.ncg, node2.ncg))
    else:
        print('{:14} {:14.4} {:14.4}'.format('SSI', node1.ssi, node2.ssi))
    
    #pipeline
    print('\n{}'.format('Pipeline Section'))
    print('{:18} {:14.4f}'.format('Length', pipe.length))
    print('{:18} {:14.4f}'.format('Total F_Loss', pipe.f_tot))
    print('{:18} {:14.4f}'.format('  Skin Friction', pipe.f_skin))
    print('{:18} {:14.4f}'.format('  Fittings Loss', pipe.f_fit))
    print('{:18} {:14.4f}'.format('  Expansion', pipe.f_exp))
    print('{:18} {:14.4f}'.format('  Contraction', pipe.f_con))

    #print('{:18} {:14.4f}'.format('h_out (J/s)', pipe.h_out))
    print('{:18} {:14.4f}'.format('Heat Loss (J/kg)', pipe.Q))
    print('{:18} {:14.4f}'.format('gdz', pipe.gdz))
    print('{:18} {:14.4f}'.format('udu', pipe.udu))
    print('{:18} {:14.4f}'.format('Total Loss', pipe.t_loss))
    print('\n')

In [5]:
'''Main Function to calculate Pressure drop along the steamline'''
def calc_dp (node_in, node_out, pipeline, p_f):
    #print(node_dict['S501'],'\n',node_dict['S501X'],'\n',pipe_dict['STL501'])
    #print (pipeline[0:2])
    #print(pipeline, pipe_dict[pipeline].is_stl())
    '''Calculate Steam T_sat, Rho and Mu from steam table'''
    if pipe_dict[pipeline].is_stl():
        node_dict[node_in].steam_p()
    else:
        node_dict[node_in].brine_t()
        
    print(pipeline, pipe_dict[pipeline].is_stl(), p_f)

    #node_dict[node_in].steam_p()
    #print(node_dict[node_in].t)
    
    '''Calculate area and velocity of inlet node; Only area for outlet node'''
    node_dict[node_in].calc_area()
    node_dict[node_in].calc_velocity(node_dict[node_in])
    node_dict[node_out].calc_area()
    node_dict[node_out].calc_velocity(node_dict[node_in])
    #print('{} = {:.4f}'.format(node_dict[node_in].ID, node_dict[node_in].v))
    
    '''Calculate Relative Roughness of the pipeline'''
    pipe_dict[pipeline].rel_roughness(node_dict[node_in].d)
    #print('{:.8f}'.format(pipe_dict[pipeline].rel_rough))
    
    '''Calculate Reynolds Number at the inlet node'''
    node_dict[node_in].nRE()
    #print('{:.0f}'.format(node_dict[node_in].reynolds))
    
    '''Calculate friction factor of the pipe'''
    if node_dict[node_in].reynolds > 0: 
        pipe_dict[pipeline].ff_fanning(node_dict[node_in].reynolds)
        #print('{}'.format(pipe_dict[pipeline].ff))
    else:
        pass
    
    '''Calculate skin friction along the pipe'''
    if node_dict[node_in].reynolds > 0: 
        pipe_dict[pipeline].skin_friction(node_dict[node_in].v, node_dict[node_in].d)
        #print(pipe_dict[pipeline].ff, pipe_dict[pipeline].length, node_dict[node_in].v, node_dict[node_in].d)
        #print('{}'.format(pipe_dict[pipeline].f_skin))
    else:
        pipe_dict[pipeline].f_skin = 0
    
    '''Calculate expansion or contraction friction loss'''
    if node_dict[node_in].area == node_dict[node_out].area:
        pipe_dict[pipeline].f_exp = 0
        pipe_dict[pipeline].f_con = 0
    
    if node_dict[node_in].area < node_dict[node_out].area:
        pipe_dict[pipeline].f_con = 0
        pipe_dict[pipeline].f_expansion(node_dict[node_in].area, node_dict[node_out].area, node_dict[node_in].v)
    
    else:
        pipe_dict[pipeline].f_exp = 0
        pipe_dict[pipeline].f_contraction(node_dict[node_in].area, node_dict[node_out].area, node_dict[node_in].v)
    #print('exp cont',pipe_dict[pipeline].f_exp, pipe_dict[pipeline].f_con, ' In/Out', \
    #     node_dict[node_in].d, node_dict[node_out].d)
    
    '''Calculate Friction Loss from fittings'''
    pipe_dict[pipeline].f_fittings(kf_list, node_dict[node_in].v)
    #print(pipe_dict[pipeline].f_fit)
    
    '''Calcutate total friction loss'''
    pipe_dict[pipeline].f_total()
    #print('Total Friction Loss: ',pipe_dict[pipeline].f_tot)
    
    '''Calculate Heat Loss'''
    pipe_dict[pipeline].q_loss(node_dict[node_in], insul)
    #print('heat Loss: ', pipe_dict[pipeline].Q , pipe_dict[pipeline].length)
    #print(pipeline, pipe_dict[pipeline].is_stl(), node_dict[node_in].rho)
    '''Calculate gdz and udu'''
    pipe_dict[pipeline].f_gdz(node_dict[node_in].z, node_dict[node_out].z)
    pipe_dict[pipeline].f_udu(node_dict[node_in].v, node_dict[node_out].v)
    #print(pipe_dict[pipeline].gdz, pipe_dict[pipeline].udu)
    
    '''Total Loss'''
    pipe_dict[pipeline].total_loss()
    
            
    '''Calculate Outlet Pressure'''
    if pipe_dict[pipeline].is_stl():
        node_dict[node_out].ps_out(node_dict[node_in], pipe_dict[pipeline])
        #print('P-out = ', node_dict[node_in].p, node_dict[node_out].p)
    else:
        node_dict[node_out].pb_out(node_dict[node_in], pipe_dict[pipeline])
        #print('P-out = ', node_dict[node_in].p, node_dict[node_out].p, node_dict[node_in].rho)
        
    '''Calculate mass Flow of Outlet Node'''
    if pipe_dict[pipeline].is_stl():
        node_dict[node_out].calc_mf(node_dict[node_in], pipe_dict[pipeline], p_f)
    else:
        node_dict[node_out].brine_mf(node_dict[node_in], pipe_dict[pipeline], p_f)

    
    '''Calc T-out for brine line'''
    if pipe_dict[pipeline].is_stl():
        pass
    else:
        node_dict[node_out].tb_out(node_dict[node_in],pipe_dict[pipeline])
    
    '''Calculate Exergy at Inlet and Outlet'''
    node_dict[node_out].calc_exergy()
    node_dict[node_in].calc_exergy()
    
    '''Calculate SiO2 at Outlet Node'''
    if pipe_dict[pipeline].is_stl():
        node_dict[node_out].sio2 = 0
    else:
        node_dict[node_out].sio2_out(node_dict[node_in])
    
    '''Calculate SSI of outlet node'''
    if pipe_dict[pipeline].is_stl():
        node_dict[node_out].ssi = 0
    else:
        node_dict[node_out].calc_ssi()
        
    '''Calculate CO2 and H2S of steamline outlet node'''
    if pipe_dict[pipeline].is_stl():
        node_dict[node_out].co2_h2s(node_dict[node_in], p_f)
    else:
        node_dict[node_out].co2 = 0
        node_dict[node_out].h2s = 0
    
    '''Calculate NCG for steam lines'''
    if pipe_dict[pipeline].is_stl():
        node_dict[node_out].calc_ncg()
    else:
        node_dict[node_out].ncg = 0
    
    '''Print Node and Pipe Parameters'''
    print_f(node_dict[node_in], node_dict[node_out], pipe_dict[pipeline])